In [1]:
import os
import csv
import cv2
import imutils
import easyocr
import webbrowser
import numpy as np
import pandas as pd
import tensorflow as tf
import pytesseract as pt
import plotly.express as px
import matplotlib.pyplot as plt
import xml.etree.ElementTree as xet


from glob import glob
from skimage import io
from shutil import copy
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

/Users/rozidatululummiah/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model = tf.keras.models.load_model('/Users/rozidatululummiah/Downloads/Tensorflow/models/research/object_detection.h5')
print('Model loaded Sucessfully')

Model loaded Sucessfully


In [3]:
def object_detection(path):
    
    # Read image
    image = load_img(path) # PIL object
    image = np.array(image,dtype=np.uint8) # 8 bit array (0,255)
    image1 = load_img(path,target_size=(224,224))
    
    # Data preprocessing
    image_arr_224 = img_to_array(image1)/255.0 # Convert to array & normalized
    h,w,d = image.shape
    test_arr = image_arr_224.reshape(1,224,224,3)
    
    # Make predictions
    coords = model.predict(test_arr)
    
    # Denormalize the values
    denorm = np.array([w,w,h,h])
    coords = coords * denorm
    coords = coords.astype(np.int32)
    
    # Draw bounding on top the image
    xmin, xmax,ymin,ymax = coords[0]
    pt1 =(xmin,ymin)
    pt2 =(xmax,ymax)
    cv2.rectangle(image,pt1,pt2,(0,255,0),3)
    xmin ,xmax,ymin,ymax = coords[0]
    roi = img[ymin:ymax,xmin:xmax]
    fig = px.imshow(roi)
    return text_2




In [4]:
def save_results(text1, text2, csv_filename):
    with open(csv_filename, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow([text1], [text2])

In [5]:
def read_plat(images):
    import imutils
    img = cv2.imread(images)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) #Grayscale and Blur
    #Apply filter and find edges for localization
    bfilter = cv2.bilateralFilter(gray, 11, 17, 17) #Noise reduction
    edged = cv2.Canny(bfilter, 30, 200) #Edge detection
    # Find Contours and Apply Mask
    keypoints = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(keypoints)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:10]
    
    location = None
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 10, True)
        if len(approx) == 4:
            location = approx
            break
            
    mask = np.zeros(gray.shape, np.uint8)
    new_image = cv2.drawContours(mask, [location], 0,255, -1)
    new_image = cv2.bitwise_and(img, img, mask=mask)
    
    (x,y) = np.where(mask==255)
    (x1, y1) = (np.min(x), np.min(y))
    (x2, y2) = (np.max(x), np.max(y))
    cropped_image = gray[x1:x2+1, y1:y2+1]
    
    #use easy ocr to read text
    reader = easyocr.Reader(['en'])
    result = reader.readtext(cropped_image)
    text = result[0][-2]
    return text

In [6]:
import csv

def cek(csv_filename, count):
    with open(csv_filename, mode='r', newline='') as f:
        csv_reader = csv.reader(f)
        rows = list(csv_reader)

        if len(rows) >= 2:  # Memastikan minimal ada dua baris untuk dibandingkan
            text1 = rows[0][count]  # Mengambil nilai dari baris pertama
            text2 = rows[1][count] # Mengambil nilai dari baris kedua

            # Membandingkan nilai dari kedua baris
            if text1 == text2:
                return print("Barcode STNK dengan Plat No sesuai, Silahkan lewat!")
            else:
                return print("Barcode STNK dengan Plat No tidak cocok!")
        else:
            return print("Tidak cukup baris untuk dibandingkan.")

            



In [7]:
harcascade = "model/haarcascade_russian_plate_number.xml"
detector = cv2.QRCodeDetector()
cap = cv2.VideoCapture(0)

cap.set(3, 640)  # width
cap.set(4, 480)  # height

min_area = 500
count = 0

text_2 = None  # Initialize text_2 outside the loop

while True:
    success, img = cap.read()
    data, _, _ = detector.detectAndDecode(img)

    plate_cascade = cv2.CascadeClassifier(harcascade)
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    plates = plate_cascade.detectMultiScale(img_gray, 1.1, 4)

    cv2.imshow("Result", img)
    for (x, y, w, h) in plates:
        area = w * h

        if area > min_area:
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(img, "Number Plate", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)

            img_roi = img[y: y + h, x:x + w]
            cv2.imshow("ROI", img_roi)
            cv2.putText(img, "Number Plate Detected", (x, y - 5), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (255, 0, 255), 2)
            cv2.imwrite("detected_plates/scaned_img_" + str(count) + ".jpg", img_roi)
            break
        images = "detected_plates/scaned_img_" + str(count) + ".jpg"  # Assign the value to 'images'
        count += 1

    if data:
        if data.startswith('http') or data.startswith('https'):
            # Jika QR code berisi URL, buka di web browser
            webbrowser.open(data)
            break
        else:
            # Jika QR code berisi teks, tampilkan teksnya
            text_2 = data
            break

    cv2.imshow("Result", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to quit the loop
        break

# Release the camera and close all windows when 'q' is pressed
cap.release()
cv2.destroyAllWindows()

# Check if 'images' is defined before calling object_detection
if 'images' in locals():
    text_1 = object_detection(images)
    save_results(text_1, text_2, 'plat_nom.csv')
else:
    print("No license plate image detected.")

2023-11-20 17:54:38.845 Python[27120:915693] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


No license plate image detected.


In [10]:
cek("plat_nom.csv", count-1)

Tidak cukup baris untuk dibandingkan.
